In [8]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU,LeakyReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam,SGD

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

#Creating Model
def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,strides= (1 if not downsample else 2),filters=filters,padding="same",kernel_initializer = glorot_uniform(seed=0))(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,strides=1,filters=filters,padding="same",kernel_initializer = glorot_uniform(seed=0))(y)

    if downsample:
        x = Conv2D(kernel_size=1,strides=2,filters=filters,padding="same",kernel_initializer = glorot_uniform(seed=0))(x)

    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,strides=1,filters=num_filters,padding="same",kernel_initializer = glorot_uniform(seed=0))(t)
    t = relu_bn(t)
    
    num_blocks_list = [2,2, 2,2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    t=Dense(512,activation='relu')(t)
    t=BatchNormalization()(t)
    outputs = Dense(100, activation='softmax')(t)
    
    model = Model(inputs, outputs)
    adam=Adam(learning_rate=0.001,clipnorm=1,name='adam')
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

    return model

In [12]:
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping

(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()
#x_train = x_train.astype('float32') / 255
#x_test = x_test.astype('float32') / 255
from keras.utils import to_categorical
y_train = to_categorical(Y_train,100)
y_test = to_categorical(Y_test,100)

model = create_res_net() 
model.summary()

from keras.preprocessing.image import ImageDataGenerator
aug_data=ImageDataGenerator(
        rotation_range=20,     #randomly rotate images in the range (20 degrees)
        horizontal_flip=True,  #randomly flip images
        width_shift_range=0.1, #randomly shift images horizontally (fraction of total width)
        shear_range = 0.2,     #Shear Intensity (Shear angle in counter-clockwise direction in degrees)
        height_shift_range=0.1,#randomly shift images vertically (fraction of total height)
        zoom_range=0.2,        #Range for random zoom
        brightness_range = (0.5, 1.5))   #Range for picking a brightness shift value
aug_data.fit(x_train)

# save model after each epoch
checkpoint = ModelCheckpoint("ResNet_BatchNorm_Adam.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto',restore_best_weights=True)
hist=model.fit(aug_data.flow(x_train, y_train, batch_size=128), batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test),callbacks=[early,checkpoint])

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 32, 32, 3)    12          input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_86 (Conv2D)              (None, 32, 32, 64)   1792        batch_normalization_57[0][0]     
__________________________________________________________________________________________________
re_lu_51 (ReLU)                 (None, 32, 32, 64)   0           conv2d_86[0][0]                  
_______________________________________________________________________________________